<a href="https://colab.research.google.com/github/ovbystrova/dpl/blob/master/notebooks/Current.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
# !git clone https://github.com/ovbystrova/dpl.git
import os
os.chdir('/content/dpl')

In [2]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
%%capture
!pip install wandb
!wandb login

In [0]:
import configuration as cfg
DATA_PATH = r"data"
wandb_config = {
    'gan' : 'relgan',
    'batch_size' : cfg.BATCH_SIZE,
    'test_samples' : 200,
    'temperatur_update' : cfg.TEMP_ADPT,
    'texts_size' : '20k',
    'max_temperature' : cfg.TEMPERATURE,
    'mle_train_epochs' : cfg.MLE_train_epoch,
    'adversarial_epochs' : cfg.ADV_train_epoch,
    'clip_norm' : cfg.CLIP_NORM,
    'discriminator' : 'relgan'
}

RUN_ID = 'gan_003'

In [5]:
import wandb
wandb.init(id=RUN_ID, config=wandb_config, project='dpl')

W&B Run: https://app.wandb.ai/2ispany3/dpl/runs/gan_003

# Dataloader

In [0]:
# import os
# os.chdir('..')
from utils.dataloader import GANDataset, GenDataIter

In [0]:
DATA_PATH = r"data"
BATCH_SIZE = 64

In [5]:
iterator = GenDataIter(data=DATA_PATH,
                            batch_size=wandb_config['batch_size'],
                            shuffle=None)

total tokens:  6333


# Generator

In [0]:
# import os
# os.chdir('..')

In [0]:
import torch
from models.RelGAN_G import RelGAN_G
MAX_SEQ_LEN = 50
PAD_IDX = 0
VOCAB_SIZE = 6330
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if_cuda = True if torch.cuda.is_available() else False

In [0]:
gen = RelGAN_G(mem_slots=1, num_heads=2, head_size=256, embedding_dim=32, hidden_dim=32,
                            vocab_size=VOCAB_SIZE, max_seq_len=MAX_SEQ_LEN, padding_idx=PAD_IDX, gpu=if_cuda)

# Discriminator

In [0]:
from models.RelGAN_D import RelGAN_D
dis = RelGAN_D(embed_dim=64, 
               max_seq_len=MAX_SEQ_LEN,
               num_rep=64, 
               vocab_size=VOCAB_SIZE, 
               padding_idx=PAD_IDX, 
               weights = 'uniform',
               gpu=if_cuda, 
               dropout=0.25)

# Instructor

In [0]:
# import os
# os.chdir('..')
from models.instructor import RelGANInstructor

In [0]:
instructor = RelGANInstructor()

# Checking

In [8]:
instructor._run()

Starting GENERATOR MLE TRAINING
[MLE-GEN] epoch 0 : pre_loss = 4.0184, BLEU-[2, 3, 4, 5] = [0.461, 0.159, 0.072, 0.045], NLL_gen = 3.467, NLL_div = 3.5955, Self-BLEU-[2, 3, 4] = [0.494, 0.188, 0.081]
[MLE-GEN] epoch 1 : pre_loss = 3.4076, BLEU-[2, 3, 4, 5] = [0.492, 0.19, 0.084, 0.05], NLL_gen = 3.2453, NLL_div = 3.1707, Self-BLEU-[2, 3, 4] = [0.517, 0.207, 0.086]
[MLE-GEN] epoch 2 : pre_loss = 3.1981, BLEU-[2, 3, 4, 5] = [0.491, 0.199, 0.09, 0.054], NLL_gen = 3.1474, NLL_div = 3.2403, Self-BLEU-[2, 3, 4] = [0.53, 0.22, 0.098]
[MLE-GEN] epoch 3 : pre_loss = 3.0645, BLEU-[2, 3, 4, 5] = [0.511, 0.205, 0.091, 0.054], NLL_gen = 3.0768, NLL_div = 2.8044, Self-BLEU-[2, 3, 4] = [0.557, 0.24, 0.104]
[MLE-GEN] epoch 4 : pre_loss = 2.9741, BLEU-[2, 3, 4, 5] = [0.52, 0.222, 0.092, 0.052], NLL_gen = 3.0494, NLL_div = 3.0141, Self-BLEU-[2, 3, 4] = [0.592, 0.271, 0.117]
[MLE-GEN] epoch 5 : pre_loss = 2.8991, BLEU-[2, 3, 4, 5] = [0.527, 0.211, 0.089, 0.053], NLL_gen = 2.9906, NLL_div = 2.8695, Self-B

Process ForkPoolWorker-351:
Process ForkPoolWorker-349:
Process ForkPoolWorker-352:
Process ForkPoolWorker-350:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/content/dpl/metrics/bleu.py", line 85, in cal_bleu
    smoothing_function=SmoothingFunction().method1)
 

KeyboardInterrupt: ignored

In [6]:
instructor.adv_train_generator(1)

0.6929979920387268

In [7]:
instructor.adv_train_discriminator(5)

0.6929468870162964

In [9]:
# instructor.gen.sample()
from utils.dataloader import GANDataset, GenDataIter
from utils.preprocess import *
SAMPLES_NUM = 100
BATCH_SIZE = 64

eval_samples = instructor.gen.sample(SAMPLES_NUM, 4 * BATCH_SIZE)
gen_tokens = tensor_to_tokens(eval_samples, instructor.idx2word_dict)
for el in gen_tokens:
    print(el)
    print()

['at', 'least', 'at', 'a', 'brilliant', 'estadio', 'maracaná', '.']

['general', 'manager', 'anton', 'rayney', 'has', 'quickly', 'its', 'seemingly', 'breakfast', 'made', 'by', 'an', 'off-duty', 'results', 'manager', 'marco', 'and', 'carries', 'in', 'this', 'weekend', ',', 'a', 'new', 'york', 'athletic', 'police', 'officer', ',', 'where', 'no', 'feature', 'by', 'medical', 'attack', 'in', '2007', 'in', 'new', 'york', 'city', 'on', 'thursday', '.']

['(', 'cnn', ')', '--', 'mitt', 'romney', 'will', 'officially', 'led', 'by', 'the', 'wake', 'of', 'a', 'hearing', 'in', 'a', 'cold', '.']

['austin', '--', 'marissa', 'rafael', 'cruzgrew', ':', '4', 'a.m.', ',', 'mass', 'tore', 'through', 'his', 'car', 'and', 'colder', 'days', ',', 'five', 'months', 'that', 'i', 'called', 'the', 'equally', 'medindie', 'gabbert', 'and', 'careers', 'oversight', 'and', 'a', 'handful', 'of', 'playstation', 'vr', 'is', 'you', 'ready', 'how', 'to', 'be', 'rather', 'than', '$', '1', 'million', 'in', 'the', 'gm']

['e